In [1]:
import tensorflow as tf

In [2]:
class FL(tf.keras.losses.Loss):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def call(self, y_true, y_pred):
        p = tf.nn.sigmoid(y_pred)
        ce = tf.nn.sigmoid_cross_entropy_with_logits(y_true,y_pred)
        alpha = tf.where(tf.equal(y_true,1) , self.alpha, (1.0 - self.alpha))
        loss = alpha * tf.pow((1.- p), self.gamma) * ce
        return tf.reduce_sum(loss, axis=-1)

# backbone

In [3]:
backbone = tf.keras.applications.ResNet50(include_top=False)

Metal device set to: Apple M1


2022-11-04 20:32:51.514997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-04 20:32:51.515088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
backbone.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                        

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

 ization)                       256)                                                              
                                                                                                  
 conv4_block6_2_relu (Activatio  (None, None, None,   0          ['conv4_block6_2_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block6_3_conv (Conv2D)   (None, None, None,   263168      ['conv4_block6_2_relu[0][0]']    
                                1024)                                                             
                                                                                                  
 conv4_block6_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block6_3_conv[0][0]']    
 ization)                       1024)                                                             
          

 conv5_block3_1_conv (Conv2D)   (None, None, None,   1049088     ['conv5_block2_out[0][0]']       
                                512)                                                              
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, None, None,   2048       ['conv5_block3_1_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block3_1_relu (Activatio  (None, None, None,   0          ['conv5_block3_1_bn[0][0]']      
 n)                             512)                                                              
                                                                                                  
 conv5_block3_2_conv (Conv2D)   (None, None, None,   2359808     ['conv5_block3_1_relu[0][0]']    
          

In [5]:
## conv2_block3_out, conv3_block4_out, conv4_block6_out, conv5_block3_out

In [6]:
output2 = backbone.get_layer('conv3_block4_out').output
output3 = backbone.get_layer('conv4_block6_out').output
output4 = backbone.get_layer('conv5_block3_out').output

In [7]:
backbone_model = tf.keras.Model(inputs=backbone.input, outputs=[output2, output3, output4])

In [8]:
backbone.outputs

[<KerasTensor: shape=(None, None, None, 2048) dtype=float32 (created by layer 'conv5_block3_out')>]

# FPN

In [9]:
channel = 120

In [10]:
class Neck(tf.keras.layers.Layer):
    def __init__(self, backbone, channel):
        super().__init__()
        self.backbone = backbone
        self.c3_1x1 = tf.keras.layers.Conv2D(channel, 1, padding='same')
        self.c4_1x1 = tf.keras.layers.Conv2D(channel, 1, padding='same')
        self.c5_1x1 = tf.keras.layers.Conv2D(channel, 1, padding='same')
        self.upsampling = tf.keras.layers.UpSampling2D(2)
        
    def call(self, images, training=False):
        c3out,c4out,c5out = self.backbone(images, training=training)
        s3 = self.c3_1x1(c3out)
        s4 = self.c4_1x1(c4out)   
        s5 = self.c5_1x1(c5out)
        
        s4 = s4 + self.upsampling(s5)
        s3 = s3 + self.upsampling(s4)        
        return s3, s4, s5

In [11]:
def head(output, anchor, channel):
    head = tf.keras.Sequential([
        tf.keras.layers.Conv2D(256, 3, padding='same', input_shape=(None,None,channel)),
        tf.keras.layers.Conv2D(256, 3, padding='same'),
        tf.keras.layers.Conv2D(256, 3, padding='same'),
        tf.keras.layers.Conv2D(256, 3, padding='same'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(output*anchor, 3)
    ])
    return head

In [12]:
neck = Neck(backbone_model, 120)
img = tf.keras.preprocessing.image.load_img('guga.jpg')
img = img.resize((224,224))
img = tf.keras.preprocessing.image.img_to_array(img)
s = neck(img[tf.newaxis])

In [13]:
class RetinaNet(tf.keras.Model):
    def __init__(self, backbone, output, anchor, channel):
        super().__init__()
        self.neck = Neck(backbone, channel)
        self.cls = head(output, anchor, channel)
        self.reg = head(4, anchor, channel)
        
    def call(self, images, training=False):
        s3,s4,s5 = self.neck(images)
        c1 = self.cls(s3)
        c2 = self.cls(s4)
        
        c3 = self.cls(s5)
        r1 = self.reg(s3)
        
        r2 = self.reg(s4)
        r3 = self.reg(s5)
        return c1,c2,c3, r1,r2,r3

In [14]:
retina = RetinaNet(backbone_model, 9, 4, 64)

In [15]:
retina(img[tf.newaxis])

(<tf.Tensor: shape=(1, 26, 26, 36), dtype=float32, numpy=
 array([[[[ 5.05758286e-01, -1.22976571e-01,  1.60448611e+00, ...,
            2.62611389e-01,  7.11411476e-01,  1.52708721e+00],
          [ 1.10899830e+00,  1.43752933e-01,  2.45885897e+00, ...,
           -6.46435857e-01,  1.25711203e+00, -3.64390612e-01],
          [ 1.21015656e+00, -3.72361660e-01,  1.13722742e+00, ...,
           -2.72444695e-01,  1.16570818e+00, -4.10287142e-01],
          ...,
          [ 1.23330677e+00,  1.20933187e+00, -5.48827767e-01, ...,
            1.50389683e+00,  1.24376380e+00, -3.49715948e-02],
          [-1.23379409e-01,  1.92856359e+00,  8.06296885e-01, ...,
            1.09461808e+00, -1.75740868e-02,  3.86929035e-01],
          [-1.02479386e+00,  1.84298503e+00,  1.23350549e+00, ...,
            5.96244931e-01,  9.19986427e-01,  5.88303447e-01]],
 
         [[ 7.19984770e-01, -2.39323318e-01,  9.61994648e-01, ...,
            8.66605639e-02,  8.46143246e-01,  1.64284408e+00],
          [ 6.